# Projet Optimisation (groupe 6) - Benjamin Piet & Damien Capéraa

## 2. Etude et resolution numerique

### Question 1

Notre problème d'optimisation est le suivant : 
$$\min_{C^Tp - d}\frac{1}{2}p^TAp-b^Tp$$\
On note $f(p)=\frac{1}{2}p^TAp-b^Tp$ la fonction coût et $c(p)=C^Tp-d$ les contraintes.
Dans la suite, on notera $x = p$ pour éviter toute confusion entre la variable des prix $p$ et les directions de descente.

##### Etude du problème
La fonction f est **fortement convexe** car la matrice $A$ est symétrique definie positive. Pour le démontrer, on peut déjà remarquer que $A$ est symétrique puis utiliser le théorème d'éuivalence $A \in S_n^{++}(\mathbb{R}) \Leftrightarrow A \in S_n(\mathbb{R}) \, et \, Sp(A) \subset \mathbb{R}^{+*}$ et calculer les valeurs propres de A avec Python :

In [9]:
import numpy as np
A = np.array([[3.0825, 0, 0, 0], [0, 0.0405, 0, 0], [0, 0, 0.0271, -0.0031], \
     [0, 0, -0.0031, 0.0054]])
valeurs_propres = np.linalg.eig(A)[0]
print(f"Liste des valeurs propres de A : {valeurs_propres}")

Liste des valeurs propres de A : [0.02753417 0.00496583 3.0825     0.0405    ]


Toutes les valeurs propres de $A$ étant strictement positives, elle est symétrique définie positive. Par conséquent, la fonction de coût quadratique f est convexe.\
De plus, l'ensemble de recherche est convexe, car les contraintes sont toutes affines.
On a donc **l'existence d'un unique minimum global**.\
\
Le conditionnement du problème vaut :
$$K(A) = \frac{max(Sp(A))}{min(Sp(A))} \approx 620,7$$
ce qui est assez élevé. On doit donc s'attendre à ce qu'une petite erreur dans les données du problème engendre une grande erreur dans la solution estimée numériquement.

##### Choix d'une méthode de résolution
* Les contraintes de notre problème sont des contraintes d'inégalités. Nous proposons d'utiliser un algorithme de recherche du point selle du Lagrangien. En effet, un point $(x^*, \lambda ^*)$ est un point selle du Lagrangien si et seulement si $x^*$ est solution du problème d'otpimisation.
* En outre, si l'on suppose que les contraintes actives en $x^*$ sont qualifiées, alors la convexite de $f$ garantit l'existence d'un point selle du Lagrangien grâce au théorème 29 du cours.
* Finalement, nous proposons d'implémenter **l'algorithme d'Uzawa**, qui repose bien sur le concept de dualité du problème (p.38 du polycopié) : à partir de $x^0 \in \mathbb{R}^4$, $\lambda^0 \in \mathbb{R}^7$, $\rho \in \mathbb{R}^+$ quelconques, on note $\mathbb{R}^7 \ni \lambda \mapsto P(\lambda) \in (\mathbb{R}^+)^7$ la projection sur $(\mathbb{R}^+)^7$, itérer : 
    * résoudre $\min_{x \in \mathbb{R}^4} \mathcal{L}(x, \lambda ^k)$, on note $x^{k+1}$ la solution ;
    * $\lambda ^{k+1} = P(\lambda ^k + \rho c(x^{k+1}))$.
* Pour la partie minimisation, nous allons utiliser deux algorithmes : un algorithme de gradient à pas optimal, en déterminant un pas à chaque itération à l'aide des conditions de Wolfe, et un algorithme de gradient à pas fixe. Nous déterminerons la valeur du pas fixe en observant les pas optimaux choisis avec les condition de Wolfe.
* Pour la partie maximisation, nous gardons la méthode proposée dans l'algorithme qui s'apparente à une montée de gradient à pas fixe.
* En parallèle, nous allons utiliser la fonction `minimize` du module `scipy.optimize` de Python pour pouvoir apprécier l'efficacité de nos algorithmes.

### Question 2

In [10]:
# Mise en place des fonctions et valeurs numériques
import numpy as np

A = np.array([[3.0825, 0, 0, 0], [0, 0.0405, 0, 0], [0, 0, 0.0271, -0.0031], \
    [0, 0, -0.0031, 0.0054]])
b = np.array([[2671], [135], [103], [19]])
C = np.array([[-0.0401, -0.1326, 1.5413, 0.0, 0.0, 0.0, 0.0160], \
    [-0.0162, -0.0004, 0.0, 0.0203, 0.0, 0.0, 0.0004], \
    [-0.0039, -0.0034, 0.0, 0.0, 0.0136, -0.0016, 0.0005], \
    [0.0002, 0.0006, 0.0, 0.0, -0.0015, 0.0027, 0.0002]])
d = np.array([[-92.6], [-29.0], [2671], [135], [103], [19], [10]])

def f(p):
    return float(0.5 * np.matmul(p.T, np.matmul(A, p)) - np.dot(b.T, p))

def c(p):
    return np.matmul(C.T, p) - d

def grad_f(p):
    return np.matmul(A, p) - b

def grad_c(p):
    return C

lambda0 = np.ones((7, 1))

##### Estimation de la solution avec `scipy.optimize.minimize`

In [11]:
from scipy.optimize import minimize
from scipy.optimize import LinearConstraint
import numpy as np

# Réécriture des contraites sous une forme compréhensible par scipy.optimize.minimize :
linear_constraint = LinearConstraint([[-0.0401, -0.0162, -0.0039, 0.0002], \
    [-0.1326, -0.0004, -0.0034, 0.0006], [1.5413, 0, 0, 0], \
    [0, 0.0203, 0, 0], [0, 0, 0.0136, -0.0015], \
    [0, 0, -0.0016, 0.0027], [0.0160, 0.0004, 0.0005, 0.0002]], \
    [-np.inf]*7, [-92.6, 29.0, 2671, 135, 103, 19, 10])

x0 = np.ones((4,))
print("scipy.optimize.minimize...")
res = minimize(f, x0, method ='trust-constr', constraints=[linear_constraint], \
    options={'verbose': 1})
print(f"Solution estimée : {res.x}")

scipy.optimize.minimize...
`xtol` termination condition is satisfied.
Number of iterations: 107, function evaluations: 745, CG iterations: 171, optimality: 1.87e-05, constraint violation: 0.00e+00, execution time: 0.13 s.
Solution estimée : [ 420.23259847 4025.00827192 2774.95770729 1393.98131029]


On trouve donc un prix en €/tonnes optimal de :
* **420** pour le **lait**
* **4025** pour le **beurre**
* **2775** pour le **gouda**
* **1394** pour le **edam**

Ce qui correspond aux quantités :
* 2023 tonnes pour le lait
* 53.3 tonnes pour le beurre
* 67.4 tonnes pour le gouda
* 19.9 tonnes pour l'edam
\
**Remarque** : Nous ne sommes pas totalement sûrs au niveau des unités.

En fait, après avoir testé avec la fonction `scipy.optimize` pour avoir une idée de la solution attendue, nous avons trouvé que l'algorithme à pas fixe semble mieux converger que celui à pas de Wolfe, et en moins de temps (même si le nombre d'iteration est bien plus grand, le temps d'execution est plus faible).\
Nous mettons tous de même les deux algorithmes, mais nous vous conseillons de ne pas lancer celui à pas de Wolfe

##### Minimisation avec algorithme de gradient à pas fixe

In [12]:
import time

def uzawa_fixed_step(fun, grad_fun, c, grad_c, x0, l, rho, lambda0, max_iter = 1000000, epsilon_grad_L = 1e-3):
    ''' epsilon_grad_L est un paramètre qui fixe une borne supérieure
    du gradient du Lagrangien en guise de condition d'arrêt.
    l est le pas fixe de descente (selon x).
    rho est le pas fixe de montée (selon lambda).
    '''
    k = 0
    xk = x0
    lambdak = lambda0
    grad_Lagrangien_xk = grad_fun(xk) + np.matmul(grad_c(xk), lambdak)
    while ((k < max_iter) and (np.linalg.norm(grad_Lagrangien_xk) > epsilon_grad_L)):
        grad_Lagrangien_xk = grad_fun(xk) + np.matmul(grad_c(xk), lambdak)
        pk = -grad_Lagrangien_xk
        xk = xk + l*pk
        lambdak = np.array([[max(0, lambdak[i, 0] + rho*c(xk)[i, 0])] for i in range(7)]) # projection sur R+^7
        k = k + 1
    print(f"Nombre d'iterations : {k}")
    print(f"lambdak : ", lambdak)
    return xk

x0 = np.ones((4, 1))
print("Uzawa fixed step...")
start_time = time.time()
x_fixed_step = uzawa_fixed_step(f, grad_f, c, grad_c, x0, 0.5, 0.1, lambda0)
''' Note: après avoir testé l'algorithme avec conditions de Wolfe, on a un pas au début 
qui est au alentours de 0.5, puis des oscillations qui dépassent rarement l'unité. 
C'est pour cela que nous avons choisi un pas de 0.5 pour l'algorithme à pas fixe.'''
print(f"Temps d'exécution : {time.time() - start_time}")
print(f"x_fixed_step : {x_fixed_step}")
print(f"c(x_fixed_step) : {c(x_fixed_step)}")

Uzawa fixed step...
Nombre d'iterations : 537076
lambdak :  [[ 4036.34636983]
 [    0.        ]
 [    0.        ]
 [    0.        ]
 [    0.        ]
 [    0.        ]
 [95112.14898624]]
Temps d'exécution : 21.10329270362854
x_fixed_step : [[ 425.32463559]
 [4008.48593851]
 [2792.62037364]
 [1449.69225136]]
c(x_fixed_step) : [[ 5.72890218e-03]
 [-3.76265350e+01]
 [-2.01544714e+03]
 [-5.36277354e+01]
 [-6.71949013e+01]
 [-1.95540235e+01]
 [ 9.48371819e-02]]


##### Minimisation vec algorithme de gradient à pas variable
*Remarque* : cet algorithme met beaucoup de temps à s'exécuter (environ 2 minutes).

In [13]:
def wolfe_step(fun, grad_fun, xk, pk, c1 = 0.25, c2 = 0.75, M = 1000):
    ''' Calcul du pas optimal selon les conditions de Wolfe.'''
    l_moins = 0
    l_plus = 0
    f_xk = fun(xk)
    grad_f_xk = grad_fun(xk)
    li = 0.001
    i = 0
    while(i < M):
        if (fun(xk + li*pk) > (f_xk + c1*li*np.dot(grad_f_xk.T, pk))):
            # Condition d'Armijo pas respectée : on diminue le pas.
            l_plus = li
            li = (l_moins + l_plus)/2.0
        else:
            if (np.dot(grad_fun(xk + li*pk).T, pk) < c2*np.dot(grad_f_xk.T, pk)):
                # Condition de courbure pas respectée : on augmente le pas.
                l_moins = li
                if (l_plus == 0):
                    li = 2*li
                else:
                    li = (l_moins + l_plus)/2.0
            else:
                return li
        i = i + 1
    return li


def uzawa_wolfe_step(fun, grad_fun, c, grad_c, x0, rho, lambda0, max_iter = 1000000, epsilon_grad_L = 1e-3):
    k = 0
    xk = x0
    lambdak = lambda0
    grad_Lagrangienk_xk = grad_fun(xk) + np.matmul(grad_c(xk), lambdak)
    while ((k < max_iter) and (np.linalg.norm(grad_Lagrangienk_xk) > epsilon_grad_L)):
        Lagrangienk = lambda x : fun(x) + np.dot(lambdak.T, c(x))
        grad_Lagrangienk = lambda x : grad_fun(x) + np.matmul(grad_c(x), lambdak)
        grad_Lagrangienk_xk = grad_Lagrangienk(xk)
        pk = -grad_Lagrangienk_xk
        lk = wolfe_step(Lagrangienk, grad_Lagrangienk, xk, pk)
        xk = xk + lk*pk
        lambdak = np.array([[max(0, lambdak[i, 0] + rho*c(xk)[i, 0])] for i in range(7)]) # projection sur R+^7
        k = k + 1
    print(f"Nombre d'iterations : {k}")
    print(f"lambdak : {lambdak}")
    return xk

print("Uzawa wolfe step...")
start_time = time.time()
x_wolfe_step = uzawa_wolfe_step(f, grad_f, c, grad_c, x0, 0.1, lambda0)
print(f"Temps d'exécution : {time.time() - start_time}")
print(f"x_wolfe_step : {x_wolfe_step}")
print(f"c(x_wolfe_step) : {c(x_wolfe_step)}")

Uzawa wolfe step...
Nombre d'iterations :  385222
lk :  2.048
Temps d'exécution : 107.36856865882874
x_wolfe_step : [[ 437.94582898]
 [3967.55025756]
 [2836.31660775]
 [1587.31018024]]
c(x_wolfe_step) : [[ 1.98853514e-02]
 [-3.93497274e+01]
 [-1.99599409e+03]
 [-5.44587298e+01]
 [-6.68070594e+01]
 [-1.92523691e+01]
 [ 3.29773707e-01]]


##### Commentaires sur toutes les méthodes

|                         | Solution renvoyée | Nombre d'itérations | Temps d'exécution (s)|
|:-----------------------:|:-----------------:|:-------------------:|:--------------------:|
| scipy.optimize.minimize |$\begin{pmatrix} 420&4025&2775&1394\\ \end{pmatrix}$|107|0.13|
| Uzawa pas fixe          |$\begin{pmatrix} 425&4008&2793&1450\\ \end{pmatrix}$|537 076|21|
| Uzawa pas optimal       |$\begin{pmatrix} 438&3968&2836&1587\\ \end{pmatrix}$|385 222|107|

Nous avons fait fonctionné les deux algorithmes d'Uzawa avec les mêmes critères d'arrêt (nombre d'itération et norme du gradient du Lagrangien inférieure à $10^{-3}$), nous pouvons donc comparer leurs performances.
* Le module `scipy.optimize` offre clairement la meilleure performance, sur tous les points, et sans comparaison avec les performances des deux autres algorithmes.
* L'algorithme d'Uzawa avec descente de gradient à pas optimal donne des performances décevantes sur le temps d'exécution. Cela est probablement dû aux calculs générés par la recherche du pas optimal sous les conditions de Wolfe : cela rajoute plusieurs dizaines de calculs par itération dans la boucle d'Uzawa. En outre, la solution qu'il propose semble plus éloignée de la solution réelle que l'algorithme d'Uzawa à pas fixe (en regardant composante par composante, cela est assez visible). Il n'obtient de meilleure performance que sur le nombre d'exécution, avec un écart d'environ 150 000 itérations, ce qui n'est pas négligeable. Cependant, ce nombre de tient pas compte des itérations faites dans la boucle de recherche du pas optimal, ce qui évacue de fait un nombre très important de calculs.
* L'algorithme d'Uzawa à pas fixe obtient des performances meilleures que celui à pas variable, et c'est assez surprenant. Nous pouvons supposer que notre code comporte des erreurs qui rendent l'algorithme à pas optimal moins efficace que ce qu'il devrait être.

## 3. Etude avancée

### Question 1

Tout d'abord, les trois méthodes nous montrent que les multiplicateurs de Lagrange au point solution du problème $x^*$ sont nuls pour toutes les contraintes sauf la 1 et la 7, qui sont donc les deux seules contraintes actives. En utilisant le théorème Karush-Kuhn-Tucker (KKT), on a l'égalité suivante où $\lambda_2 = \lambda_3 = \lambda_4 = \lambda_5 = \lambda_6 = 0$ :
$$
\begin{align}
\nabla f(x^*) &= - \sum_{i=1}^7 \lambda_i \nabla c_i(x^*) \\
Ap^* - b &= - C\lambda \\
p^* &= A^{-1}(b - C\lambda) \\
p^* &= A^{-1}
\begin{pmatrix}
2671 + 0.0401\lambda_1 - 0.0160\lambda_7 \\
135 + 0.0162\lambda_1 - 0.0004\lambda_7 \\
103 + 0.0038\lambda_1 - 0.0005\lambda_7 \\
19 - 0.0002\lambda_1 - 0.0002\lambda_7 \\
\end{pmatrix}
\end{align}
$$

### Question 2
On pourra supposer qu'une petite variation des élasticités-prix engendre une petite variation des coefficients devant $\lambda_1$ et $\lambda_7$, et vérifie en intégrant cette solution dans l'expression des contraintes $c$ que les contraintes 1 et 7 sont toujours actives avec l'hypothèse $\lambda_1 > 0$ et $\lambda_7 > 0$ (non dégénérescence de la solution).

## Annexe

Avant de nous orienter vers l'algorithme d'Uzawa, nous avons tenté d'utiliser un algorithme des contraintes actives (p.39 du polycopié). Cet algorithme requiert en particulier de caluler les multiplicateurs de Lagrange pour chaque contrainte à chaque itération. C'est précisément cette étape que nous n'avons pas réussi à implémenter, et notre algorithme n'a malheureusement jamais abouti. Nous mettons néanmoins ici le code et les explications que nous avions produits.

**Etape 2 (a) de l'algorithme : résolution du problème d'otpimisation sous contraintes d'égalité** \
L'étape consiste à résoudre $\displaystyle \min_{c_i^Tp = 0, i \in W^k} \frac{1}{2}p^TGp + (Gx^k + d)^Tp$.
On introduit le Lagrangien du problème $\mathcal{L}(p, \lambda) = \frac{1}{2}p^TGp + (Gx^k + d)^Tp + \lambda^TD^Tp$ ou $D$ est la matrice composée des colonnes $c_i$ de C pour $i \in W^k$.
Un point stationnaire du Lagrangien sans contraintes est un point stationnaire de la fonction coût sous contraintes : on cherche donc le point $(p^*, \lambda^*)$ qui annule les deux dérivées partielles de $\mathcal{L}$.
$$
\begin{align}
\frac{\partial\mathcal{L}}{\partial\lambda}(p^*, \lambda^*) &= 0 = D^Tp \\
\frac{\partial\mathcal{L}}{\partial p}(p^*, \lambda^*) &= 0 = Ap + (Ax^k + b) + D\lambda \\
\end{align}$$
On réécrit le problème sous forme matricielle car c'est la formulation qui peut être exploitée avec la fonction numpy `np.linalg.solve`
$$
\begin{pmatrix} D^T&0 \\
A&D\\
\end{pmatrix}
\begin{pmatrix} p \\
\lambda \\
\end{pmatrix}
=
\begin{pmatrix} 0 \\
-Ax^k - b \\
\end{pmatrix}
$$

In [1]:
#Algorithme des contraintes actives
import numpy as np

A = np.array([[3.0825, 0, 0, 0], [0, 0.0405, 0, 0], [0, 0, 0.0271, -0.0031], \
     [0, 0, -0.0031, 0.0054]])
b = np.array([[2671], [135], [103], [19]])
C = np.array([[-0.0401, -0.0162, -0.0039, 0.0002], \
    [-0.1326, -0.0004, -0.0034, 0.0006], [1.5413, 0, 0, 0], \
    [0, 0.0203, 0, 0], [0, 0, 0.0136, -0.0015], \
    [0, 0, -0.0016, 0.0027], [0.0160, 0.0004, 0.0005, 0.0002]]).transpose()
d = np.array([[-92.6], [-29.0], [2671], [135], [103], [19], [10]])
eps = 1e-8 # paramètre donnant la valeur maximale des composantes
# de la direction pk pour que pk ne soit pas considérée comme nulle.

def f(p):
    return float(0.5 * np.matmul(p.transpose(), np.matmul(A, p)) - np.dot(b.transpose(), p))

def c(p):
    return np.matmul(C.transpose(), p) - d


def xk_is_solution(xk, Wk):
    '''
    Etape 1 de l'algorithme des contraintes actives.
    Détermine, grâce aux conditions KKT, si xk est une solution du problème
    d'optimisation restreint aux contraintes contenues dans Wk.
    '''
    CC = C[:, Wk]
    dd = d[Wk, :]
    rg = np.linalg.matrix_rank(CC)
    if rg == len(Wk):
        # Les contraintes actives sont qualifiées
        # Conditions de relâchement
        Wk_systeme = Wk.copy()
        for i, c_i in enumerate([CC[:, j] for j in range(len(Wk))]):
            if np.dot(c_i, xk) -  dd[i, 0] != 0:
                # Lambda_i = 0 : on retire la contrainte Wk_systeme[i]
                del Wk_systeme[Wk_systeme.index(Wk[i])]
        Lambda = np.zeros((len(Wk), 1))
        print(Wk_systeme)
        if len(Wk_systeme) != 0:
            CC_systeme = C[:, Wk_systeme]
            dd_systeme = d[Wk_systeme, :]
            Lambda_systeme = np.linalg.solve(CC_systeme.transpose(), dd_systeme)
            for j in Wk:
                if j in Wk_systeme:
                    Lambda[Wk.index(j), 0] = Lambda_systeme[Wk_systeme.index(j), 0]
        return all([Lambda[i, 0] > 0 for i in range(len(Wk))]), Lambda
    else:
        # Les contraintes actives ne sont pas qualifiées
        return False, None

W0 = [0, 1, 2, 3] # arbitraire mais permet de n'avoir qu'un point de départ possible
# car le système à résoudre contient 4 équations pour 4 inconnues.
# Recherche de p0 où ces 4 contraintes sont actives
CC = C[:, W0]
dd = d[W0, :]
x0 = np.linalg.solve(CC.transpose(), dd)

def contraintes():
    xk = x0
    Wk = W0
    while True:
        if xk_is_solution(xk, Wk)[0]:
            return xk, Wk
        # (a)
        '''
        L'étape (a) consiste à résoudre un problème d'optimisation sous
        contraintes d'égalité. On cherche alors (p*, Lambda*) point stationnaire
        du lagrangien associé. Cette recherche, comme présenté dans le notebook,
        aboutit à la résolution d'un système linéaire.
        '''
        D = C[:, Wk]
        E_ligne_0 = np.concatenate((D.transpose(), np.zeros((D.shape[1], D.shape[1]))), axis=1)
        E_ligne_1 = np.concatenate((A, D), axis=1)
        E = np.concatenate((E_ligne_0, E_ligne_1), axis=0)
        F = np.concatenate((np.zeros((D.shape[1], 1)), -np.matmul(A, xk) + b), axis=0)
        X = np.linalg.solve(E, F)
        pk = X[0:4, :]
        if any([abs(pk[i, 0]) > eps for i in range(4)]):
            # (b) : pk != 0
            W_barre = [i for i in range(7) if i not in Wk]
            L, indices = [], []
            for i in W_barre:
                c_i = C[:,i]
                if np.dot(c_i, pk) > 0:
                    L.append((d[i, 0] - np.dot(c_i, xk))/np.dot(c_i, pk))
                    indices.append(i)
            if L != []:
                alphak = min(1, min(L))
            else:
                alphak = 1
            xk = xk + alphak*pk
            if alphak < 1:
                j = indices[L.index(min(L))]
                Wk.append(j)
        else:
            # (c) ! pk = 0
            booleen, Lambda = xk_is_solution(xk, Wk)
            if booleen:
                return xk, Wk
            Lambda = Lambda.tolist()
            c = Lambda.index(min(Lambda))
            del Wk[c]
    return xk, Wk

In [2]:
contraintes()

[0, 1, 2, 3]
[0, 1, 2, 3]
[0, 1, 3]


LinAlgError: Last 2 dimensions of the array must be square